# Detekcja danych spoza rozkładu (OOD Detection – ImageNet-O)

## Instalacja bibliotek

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

!pip install pytorch_lightning torchvision codecarbon scikit-learn ood-metrics

## Pobranie danych

In [ ]:
from google.colab import drive, files
files.upload() # Służy do umieszczenia pliku kaggle.json
drive.mount('/content/drive')

!mkdir -p ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download sautkin/imagenet1kvalid
!unzip -q /content/imagenet1kvalid.zip -d /content/imagenet1kv1_valid

!mkdir -p /content/ood
!tar -xvf /content/drive/MyDrive/imagenet-o.tar -C /content/ood


## Importy i konfiguracja

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import time
import os
import subprocess
from torchvision import datasets
from torchvision.models import (
    efficientnet_v2_s, EfficientNet_V2_S_Weights,
    resnet50, ResNet50_Weights,
    densenet201, DenseNet201_Weights,
    convnext_tiny, ConvNeXt_Tiny_Weights,
    mobilenet_v3_large, MobileNet_V3_Large_Weights
)
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
import ood_metrics as om
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imagenet_path = "/content/imagenet1kv1_valid"
imagenet_ood_path = "/content/ood/imagenet-o"
batch_size = 128


## Konfiguracja modeli

In [ ]:
MODEL_CONFIGS = {
    "efficientnet_v2_s":  (efficientnet_v2_s,   EfficientNet_V2_S_Weights.IMAGENET1K_V1),
    "resnet50":           (resnet50,            ResNet50_Weights.IMAGENET1K_V1),
    "densenet201":        (densenet201,         DenseNet201_Weights.IMAGENET1K_V1),
    "convnext_tiny":      (convnext_tiny,       ConvNeXt_Tiny_Weights.IMAGENET1K_V1),
    "mobilenet_v3_large": (mobilenet_v3_large,  MobileNet_V3_Large_Weights.IMAGENET1K_V1),
}

## Funkcje pomocnicze

In [ ]:
def get_gpu_stats():
    try:
        smi = subprocess.check_output([
            "nvidia-smi", "--query-gpu=utilization.gpu,memory.used,memory.total",
            "--format=csv,noheader,nounits"
        ]).decode().strip().split("\n")[0]
        util, used, total = map(float, smi.split(","))
        return util, used, total
    except:
        return float("nan"), float("nan"), float("nan")

def make_loader(path, transform, batch_size):
    dataset = datasets.ImageFolder(path, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

def validate(model_fn, weights, loader, tag, model_name):
    model = model_fn(weights=weights).to(device).eval()

    if os.path.exists("/tmp/.codecarbon.lock"):
        os.remove("/tmp/.codecarbon.lock")
    tracker = EmissionsTracker(project_name=f"{model_name}_{tag}", log_level="error")
    tracker.start()

    if device.type == "cuda":
        torch.cuda.reset_peak_memory_stats()
    start = time.time()

    logits_list = []
    with torch.no_grad():
        for x, _ in loader:
            x = x.to(device)
            logits = model(x)
            logits_list.append(logits.cpu().numpy())

    duration = time.time() - start
    peak_mem = torch.cuda.max_memory_allocated() / 1024**2 if device.type == "cuda" else 0
    emissions = tracker.stop()
    gpu_util, gpu_mem_used, gpu_mem_total = get_gpu_stats()

    return np.vstack(logits_list), duration, peak_mem, emissions, gpu_util, gpu_mem_used, gpu_mem_total

## Pętla po modelach (InD vs OOD – ImageNet-O)

In [ ]:
results = []

for model_name, (model_fn, weights) in MODEL_CONFIGS.items():
    transform = weights.transforms()
    ind_loader = make_loader(imagenet_path, transform, batch_size)
    ood_loader = make_loader(imagenet_ood_path, transform, batch_size)

    ind_logits, ind_time, _, ind_co2, _, ind_mem_used, _ = validate(model_fn, weights, ind_loader, "InD", model_name)
    ood_logits, ood_time, _, ood_co2, _, ood_mem_used, _ = validate(model_fn, weights, ood_loader, "OOD", model_name)

    ind_msp = -F.softmax(torch.tensor(ind_logits), dim=1).max(1).values.numpy()
    ood_msp = -F.softmax(torch.tensor(ood_logits), dim=1).max(1).values.numpy()
    msp_metrics = om.calc_metrics(np.concatenate([ind_msp, ood_msp]), np.array([0]*len(ind_msp) + [1]*len(ood_msp)))

    ind_energy = -torch.logsumexp(torch.tensor(ind_logits), dim=1).numpy()
    ood_energy = -torch.logsumexp(torch.tensor(ood_logits), dim=1).numpy()
    energy_metrics = om.calc_metrics(np.concatenate([ind_energy, ood_energy]), np.array([0]*len(ind_energy) + [1]*len(ood_energy)))

    results.append({
        "model": model_name,
        "msp_auroc": msp_metrics.get("auroc", float("nan")),
        "msp_fpr@95": msp_metrics.get("fpr_at_95_tpr", float("nan")),
        "msp_det_error": msp_metrics.get("detection_error", float("nan")),
        "energy_auroc": energy_metrics.get("auroc", float("nan")),
        "energy_fpr@95": energy_metrics.get("fpr_at_95_tpr", float("nan")),
        "energy_det_error": energy_metrics.get("detection_error", float("nan")),
        "ind_time_s": ind_time,
        "ind_mem_mb": ind_mem_used,
        "ind_co2_kg": ind_co2,
        "ood_time_s": ood_time,
        "ood_mem_mb": ood_mem_used,
        "ood_co2_kg": ood_co2
    })

## Wyniki końcowe

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df